In [1]:
import cv2
#from sklearn.preprocessing import MinMaxScaler
import numpy as np
#import joblib
import tensorflow as tf
#import scipy.linalg as la
import random
#from google.colab.patches import cv2_imshow
import copy
import time
from rdkit.Chem import AllChem, Draw
from rdkit import Chem
from PIL import Image
import math
# from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem.Draw import MolDrawing, DrawingOptions
import py3Dmol
# from IPython.display import display
import matplotlib.pyplot as plt  # plt 用於顯示圖片
import matplotlib.image as mpimg  # mpimg 用於讀取圖片
import os
#import pathlib
import gradio as gr
import ipywidgets as widgets
import tkinter as tk
from tkinter import filedialog
import filetype
from IPython.display import display, clear_output

#https://blog.finxter.com/how-to-create-an-interactive-web-application-using-jupyter-notebook/

#temp = pathlib.PosixPath
#pathlib.PosixPath = pathlib.WindowsPath
out = widgets.Output()
def main(event):
    global dim4resizeimg, inputfile
    
    with out:
        clear_output()
        t1 = time.time()  # 計時開始


        # MiMaScaler = joblib.load('scalar02')  # 有數值縮放才需要，這次沒有
        root = tk.Tk()
        root.withdraw()
        file_path_storge=[]
        for i in range(2):
            file_path = filedialog.askopenfilename()
            file_path_storge.append(file_path)


        def calculatebondorder(b, eigvals, eigvecs, structure):
            # eigvecs橫式位置，縱是軌域
            # print(eigvals_list)
            eigvals_list = eigvals.real
            eigvals_list = eigvals_list.tolist()
            # print(eigvecs)
            if structure == 'ring':
                bo = [0] * b  # 環的話有碳數個鍵結
            else:
                bo = [0] * (b - 1)  # 值鏈的話有碳數-1個鍵結
            # print(bo)
            list2 = []  # list2是儲存特爭執沒有重複的元素
            for i in sorted(eigvals_list, reverse=True):
                if not i in list2:
                    list2.append(i)
            # print(list2)
            bb = copy.deepcopy(b)
            for j in range(len(list2)):
                eq_letter = np.where(np.array(eigvals_list) == list2[j])
                # print(eq_letter,'eq_letter')  # (array([2]),)
                for k in range(2):
                    for i in range(len(eq_letter[0])):
                        if bb == 0:
                            break
                        # print(bb,'bb')
                        bb = bb - 1
                        for ku in range(len(eigvecs)):
                            if ku == (len(eigvecs) - 1) and structure == 'ring':
                                bo[ku] = (eigvecs[ku][eq_letter[0][i]] * eigvecs[0][eq_letter[0][i]] + bo[ku])
                                eq_letter_list = eq_letter[0].tolist()
                                # print(ku,eq_letter[0][i],eigvecs[ku][eq_letter[0][i]],eigvecs[0][eq_letter[0][i]])

                            else:
                                # print(eigvecs,ku,eq_letter[0][i])
                                # print(eigvecs[ku][eq_letter[0][i]],'eigvecs[ku][eq_letter[0][i]]')
                                if ku != (len(eigvecs) - 1):
                                    bo[ku] = (eigvecs[ku][eq_letter[0][i]] * eigvecs[ku + 1][eq_letter[0][i]] + bo[ku])
                        #print(bo, 'bo')
                    if bb == 0:
                        break
                if bb == 0:
                    break
            return bo


        def oneordoublebond(bo, rs):
            onedouble = []
            if rs == 'straight':
                if (bo[0] - bo[1]) > 0:
                    for kj in range(len(bo)):
                        if kj % 2 == 0:
                            onedouble.append(2)
                        else:
                            onedouble.append(1)
                else:
                    for kj in range(len(bo)):
                        if kj % 2 == 0:
                            onedouble.append(1)
                        else:
                            onedouble.append(2)
            else:
                if len(bo) % 2 == 0:
                    for kj in range(len(bo)):
                        if kj % 2 == 0:
                            onedouble.append(2)
                        else:
                            onedouble.append(1)
                else:
                    boo = bo + bo
                    for kj in range(len(boo)):
                        if kj >= 1 and kj <= (len(bo) + 1):
                            #print((boo[kj] - boo[(kj + 1)]), (boo[kj] - boo[(kj - 1)]), 'boo')
                            if ((boo[kj] - boo[(kj + 1)]) > 0.1) and ((boo[kj] - boo[(kj - 1)]) > 0.1):
                                onedouble.append(2)
                            else:
                                onedouble.append(1)
                    onedouble = [onedouble[-1]] + onedouble[0:(len(onedouble) - 2)]
            #print(onedouble, 'onedouble')
            return onedouble


        def moleculevisu(back):
            back = Chem.MolToMolBlock(back)
            p = py3Dmol.view(width=500, height=500)
            p.addModel(back, 'sdf')
            p.setStyle({'sphere': {'scale': .3}, 'stick': {}})  # cartoon  stick  sphere  {'sphere':{'scale':.3},'stick':{}}
            p.render()
            #display(p.render())
            # p.zoomTo()
            # p.show()


        def cutvideotoimage(inputname, outputname, foldernamed):
            """
            輸入影片後自動截圖
            截圖完resize
            resize後變4維
            顯示圖片
            """
            import cv2
            from moviepy.editor import VideoFileClip

            clip = VideoFileClip(inputname)
            #print(clip.duration)  # seconds
            vidcap = cv2.VideoCapture(inputname)
            if not os.path.exists(foldernamed):
                os.makedirs(foldernamed)
            else:
                import shutil
                shutil.rmtree(foldernamed)
                os.makedirs(foldernamed)

            def getFrame(sec):
                vidcap.set(cv2.CAP_PROP_POS_MSEC, sec * 1000)
                hasFrames, image = vidcap.read()
                if hasFrames:
                    cv2.imwrite('./' + foldernamed + '/' + outputname + str(count) + ".png", image)  # save frame as JPG file
                return hasFrames

            sec = 0
            frameRate = int(clip.duration / 10)  # //it will capture image in each 0.5 second
            count = 1
            success = getFrame(sec)
            while success:
                count = count + 1
                sec = sec + frameRate
                sec = round(sec, 2)
                success = getFrame(sec)
            yourPath = r'./' + foldernamed
            allFileList = os.listdir(yourPath)
            d = []
            for file in allFileList:
                im = copy.deepcopy(cv2.imread(yourPath + '//' + str(file)))
                dst = cv2.resize(im, (300, 300), interpolation=cv2.INTER_AREA)
                d.append([dst])  # d=[[dst],[dst]...]
            img = np.array([i[0] for i in d])  # 變4維 img=[dst,dst....]
            plt.figure()
            for i in range(1, 10):  # https://blog.csdn.net/qwe2508/article/details/88078746
                plt.subplot(5, 2, i)
                plt.imshow(img[i - 1])
            plt.show()
            return img


        # 算階層
        def x(num):  # https://www.runoob.com/python3/python3-factorial.html
            factorial = 1
            for i in range(1, num + 1):
                factorial = factorial * i
            return factorial


        def diffCHOOMOLUMO(b, loadmodelname, structure):
            """
            "碳數=>HOMO、LUMO"

            得出Huckel矩陣
            算出特徵值、特徵向量
            製作預測HOMO、LUMO的輸入資料(可看尋練模型前的數據及如何製作)(n*n矩陣=>n*n*3*1矩陣)
              特徵向量是n*n矩陣，從裡面拿出一個軌域resize成n*1矩陣
              增加維度成1*n*1*1矩陣，存入img裡，img維度為n*n*1*1矩陣
              製作n*n*3*1矩陣(n*n*1*1矩陣=>n*n*3*1矩陣)
                要預測的軌域(從img裡拿(有順序)存入true_img(n*n*1*1矩陣拿出後，true_img為n*1*1矩陣)
                隨機選擇img裡的軌域(img[r1],img[r2])
                img[r1]以及img[r2]與true_img合併，且增加維度(1*n*3*1矩陣)存入concat_img
                concat_img合併，存入p_img，p_img維度為n*n*3*1矩陣
            拿出p_img加一維成1*n*3*1矩陣，去預測
            若預測為0表示是HOMO，則存入HOMO這個list裡
            若預測為1表示是LUMO，則存入LUMO這個list裡
            """
            # 得出Huckel矩陣
            H = np.zeros([b, b]).astype('int64')
            for j in range(b - 1):
                p1 = j + 1
                p2 = j
                H[p1, p2] = 1
                H[p2, p1] = 1
            if structure == 'ring':
                H[p1, 0] = 1
                H[0, p1] = 1
            # print(H)
            # 步驟三:算出特徵值、特徵向量
            eigvals, eigvecs = np.linalg.eig(H)
            #print(eigvals.shape, 'eigvals', eigvals)
            # print(eigvecs.shape, 'eigvecs',eigvecs)
            # 步驟5前處理
            img = []
            c = int((20 - b) / 2)
            for e in range(b):  # img=(-1,20,1,1)
                #print(eigvecs[:, e], 'eigvecs[:, e]')
                b1 = eigvecs[:, e].reshape(b, 1)
                # print(b1,'b1')
                bbb = copy.deepcopy(b1)
                bbb = copy.deepcopy(bbb[:, :, np.newaxis])
                bbb = copy.deepcopy(bbb[np.newaxis, :])
                if len(img) == 0:
                    img = copy.deepcopy(bbb)
                else:
                    img = np.concatenate((img, bbb), axis=0)
            p_img = []
            for i in range(len(img)):  # img=(-1,n,3,1)鬼域
                true_img = copy.deepcopy(img[i])
                r1 = random.randint(0, (len(img) - 1))
                r2 = random.randint(0, (len(img) - 1))
                concat_img = copy.deepcopy(np.concatenate((true_img, img[r1]), axis=1))
                concat_img = np.concatenate((img[r2], concat_img), axis=1)
                concat_img = copy.deepcopy(concat_img[np.newaxis, :])
                if len(p_img) == 0:
                    p_img = copy.deepcopy(concat_img)
                else:
                    p_img = np.concatenate((p_img, concat_img), axis=0)
            # print(p_img.shape)
            # 步驟5
            model = tf.keras.models.load_model(loadmodelname)
            HOMO, LUMO = copy.deepcopy([]), copy.deepcopy([])
            for i in range(len(p_img)):
                ss = p_img[i]
                ss2 = copy.deepcopy(ss[np.newaxis, :])
                prediction = model.predict_classes(ss2)
                #print(prediction[0])
                if prediction[0] == 0:
                    HOM = copy.deepcopy(ss[:, 1, :])  # ss[:,1,:]shape是4*1或6*1等
                    HOMO.append(HOM)
                    #print(HOM, '0')
                if prediction[0] == 1:
                    LUM = copy.deepcopy(ss[:, 1, :])
                    LUMO.append(LUM)
                    #print(LUM.shape, '0')
            bondorder = calculatebondorder(b, eigvals, eigvecs, structure)
            bond = oneordoublebond(bondorder, structure)
            return HOMO, LUMO, eigvals, eigvecs, bondorder, bond


        def HOMOLUMOstraight(dst, inputfile, structure):
            """
            "直鍊圖片的ndarry資料=>HOMO、LUMO，碳數以及特徵值、特徵向量"

            先預測輸入的四維資料
            根據輸入含有多張截圖圖片或是一張圖片判斷prediction[0]
            prediction[0]=0表示預測為2C鏈
            prediction[0]=1表示預測為4C鏈...等
            之後放入diffCHOOMOLUMO函數得出2C鏈或是4C鏈等在眾多軌域中，HOMO、LUMO，以及特徵值、特徵向量
            """
            # 步驟一前處理
            # dst = cv2.resize(im, (128,128), interpolation=cv2.INTER_AREA)
            # dst = copy.deepcopy(dst[np.newaxis, :])
            # 步驟一
            model = tf.keras.models.load_model(u'./load_model_7.1_ring_predict_eig1/predict_straight_C_number.h5')
            dst1 = model.predict(dst)
            #print(dst1.shape, 'dst1.shape')  # (13, 10) dst.shape，13張圖片10個類別
            prediction = []
            # print(len(dst1[0]),len(dst1[1]))
            if inputfile != 'image':
                #print(len(dst1[:, 0]), len(dst1[0, :]))
                for k in range(len(dst1[:, 0])):
                    for h in range(len(dst[0, :])):
                        if (dst1[k, h] >= max(dst1[k])) and max(
                                dst1[k]) >= 0.8:  # max(dst1[k]))，那一張圖片中，10個編號中哪個機率最大，dst[k,h]某個圖片的某個編號機率
                            prediction.append([dst1[k, h], h])  # [最大機率，編號]
                a = [i[1] for i in prediction]  # a13張圖片中13個最大機率的編號
                prediction[0] = max(a,
                                    key=a.count)  # https://blog.csdn.net/DeniuHe/article/details/103092390   获取list中出现次数最多的元素
            else:
                prediction = np.argmax(dst1, axis=-1)
            # prediction = model.predict_classes(dst)
            #print(prediction[0])
            # 步驟二
            if prediction[0] == 0:
                b = 2
                loadmodelname = u'./load_model_7.1_ring_predict_eig1/2.h5'
                HOMO, LUMO, eigvals, eigvecs, bondorder, bond = diffCHOOMOLUMO(b, loadmodelname, structure)
            if prediction[0] == 1:
                b = 4
                loadmodelname = u'./load_model_7.1_ring_predict_eig1/4.h5'
                HOMO, LUMO, eigvals, eigvecs, bondorder, bond = diffCHOOMOLUMO(b, loadmodelname, structure)
            if prediction[0] == 2:
                b = 6
                loadmodelname = u'./load_model_7.1_ring_predict_eig1/6.h5'
                HOMO, LUMO, eigvals, eigvecs, bondorder, bond = diffCHOOMOLUMO(b, loadmodelname, structure)
            if prediction[0] == 3:
                b = 8
                loadmodelname = u'./load_model_7.1_ring_predict_eig1/8.h5'
                HOMO, LUMO, eigvals, eigvecs, bondorder, bond = diffCHOOMOLUMO(b, loadmodelname, structure)
            if prediction[0] == 4:
                b = 10
                loadmodelname = u'./load_model_7.1_ring_predict_eig1/10.h5'
                HOMO, LUMO, eigvals, eigvecs, bondorder, bond = diffCHOOMOLUMO(b, loadmodelname, structure)
            if prediction[0] == 5:
                b = 12
                loadmodelname = u'./load_model_7.1_ring_predict_eig1/12.h5'
                HOMO, LUMO, eigvals, eigvecs, bondorder, bond = diffCHOOMOLUMO(b, loadmodelname, structure)
            if prediction[0] == 6:
                b = 14
                loadmodelname = u'./load_model_7.1_ring_predict_eig1/14.h5'
                HOMO, LUMO, eigvals, eigvecs, bondorder, bond = diffCHOOMOLUMO(b, loadmodelname, structure)
            if prediction[0] == 7:
                b = 16
                loadmodelname = u'./load_model_7.1_ring_predict_eig1/16.h5'
                HOMO, LUMO, eigvals, eigvecs, bondorder, bond = diffCHOOMOLUMO(b, loadmodelname, structure)
            if prediction[0] == 8:
                b = 18
                loadmodelname = u'./load_model_7.1_ring_predict_eig1/18.h5'
                HOMO, LUMO, eigvals, eigvecs, bondorder, bond = diffCHOOMOLUMO(b, loadmodelname, structure)
            if prediction[0] == 9:
                b = 20
                loadmodelname = u'./load_model_7.1_ring_predict_eig1/20.h5'
                HOMO, LUMO, eigvals, eigvecs, bondorder, bond = diffCHOOMOLUMO(b, loadmodelname, structure)
            # 顯示圖與結果關係

            return HOMO, LUMO, b, eigvals, eigvecs, bondorder, bond


        def HOMOLUMOring(dst, inputfile, structure):
            """
            "環圖片的ndarry資料=>HOMO、LUMO，碳數以及特徵值、特徵向量"

            先預測輸入的四維資料
            根據輸入含有多張截圖圖片或是一張圖片判斷prediction[0]
            prediction[0]=0表示預測為五環
            prediction[0]=1表示預測為六環
            之後放入diffCHOOMOLUMO函數得出五環或是六環在眾多軌域中，HOMO、LUMO，以及特徵值、特徵向量
            """
            # 步驟一前處理
            # dst = cv2.resize(im, (128,128), interpolation=cv2.INTER_AREA)
            # dst = copy.deepcopy(dst[np.newaxis, :])
            # 步驟一
            model = tf.keras.models.load_model(u'./load_model_7.1_ring_predict_eig1/predict_ring_C_number.h5')
            dst = model.predict(dst)
            #print(dst.shape, 'dst.shape')  # (13, 10) dst.shape，13張圖片10個類別
            prediction = []
            # print(len(dst[0]),len(dst[1]))
            if inputfile != 'image':
                #print(len(dst[:, 0]), len(dst[0, :]))
                for k in range(len(dst[:, 0])):
                    for h in range(len(dst[0, :])):
                        if (dst[k, h] >= max(dst[k])) and max(
                                dst[k]) >= 0.8:  # max(dst[k]))，那一張圖片中，10個編號中哪個機率最大，dst[k,h]某個圖片的某個編號機率
                            prediction.append([dst[k, h], h])  # [最大機率，編號]
                a = [i[1] for i in prediction]  # a13張圖片中13個最大機率的編號
                prediction[0] = max(a,
                                    key=a.count)  # https://blog.csdn.net/DeniuHe/article/details/103092390   获取list中出现次数最多的元素
            else:
                prediction = np.argmax(dst, axis=-1)
            # prediction = model.predict_classes(dst)
            #print(prediction[0])
            # 步驟二
            if prediction[0] == 0:
                b = 5
                loadmodelname = u'./load_model_7.1_ring_predict_eig1/5Cring.h5'
                HOMO, LUMO, eigvals, eigvecs, bondorder, bond = diffCHOOMOLUMO(b, loadmodelname, structure)
            if prediction[0] == 1:
                b = 6
                loadmodelname = u'./load_model_7.1_ring_predict_eig1/6Cring.h5'
                HOMO, LUMO, eigvals, eigvecs, bondorder, bond = diffCHOOMOLUMO(b, loadmodelname, structure)

            return HOMO, LUMO, b, eigvals, eigvecs, bondorder, bond


        def ring_or_stright(ds, inputfile):
            """
            "圖片的ndarry資料=>prediction[0]=0是直鏈，prediction[0]=1是環"

            ds是四維圖片，inputfile是圖或影片
            dsd是各個類別機率
            prediction[0]是預測出來結果
            prediction[0]=0是直鏈，prediction[0]=1是環
            """
            model = tf.keras.models.load_model(u'./load_model_7.1_ring_predict_eig1/ringorstraight.h5')
            # print(ds)
            dsd1 = model.predict(ds)
            # print(dsd.shape,'dsd.shape')  #(13,2)or(1, 2) dsd.shape
            # print(dsd)   # dsd=[[9.99898791e-01 1.01215606e-04]] dsd=[[0.9965252  0.00347477]]
            prediction = []
            # print(len(dsd[0]),len(dsd[1]))
            dsd=copy.deepcopy(dsd1)
            if inputfile != 'image':
                # print(len(dsd[:,0]),len(dsd[0,:]))
                for k in range(len(dsd[:, 0])):
                    for h in range(len(dsd[0, :])):
                        if (dsd[k, h] >= max(dsd[k])) and max(
                                dsd[k]) >= 0.8:  # max(dst[k]))，那一張圖片中，10個編號中哪個機率最大，dst[k,h]某個圖片的某個編號機率
                            prediction.append([dsd[k, h], h])  # [最大機率，編號]
                a = [i[1] for i in prediction]  # a13張圖片中13個最大機率的編號
                prediction[0] = max(a, key=a.count)  # https://blog.csdn.net/DeniuHe/article/details/103092390  获取list中出现次数最多的元素
            else:
                prediction = np.argmax(dsd, axis=-1)
            #print(prediction[0],dsd1)
            return prediction[0], {'ring': dsd1[0][1], 'line': dsd1[0][0]}


        # 判斷輸入是圖片還是影片
        #import sys
        #inputfile = str(input('image or video'))
        #inputfile = raw_input('image or video')
        #print('image or video')
        #inputfile ='image'
        #inputfile =sys.stdin.readline().strip('\n')
        """
        import ipywidgets as widgets	# 控件库
        from IPython.display import display	# 显示控件的方法
        text = widgets.Text()
        i=1
        display(text)
        inputfile=0
        def print_value(sender):
            print(sender.value)
            inputfile =str(sender)

        text.on_submit(print_value)	# 回车以提交内容
        while i==1:
            if inputfile !=0:
                i=2
        """

        dim4resizeimg=[]
        inputfile = []
        def input_file(file_path, input_number):
            global dim4resizeimg, inputfile
            #print(inputfile)
            fileURL = (file_path)
            kind = filetype.guess(fileURL) #抽出File類型
            inputfile.append(str(kind.mime).split('/')[0])
            if str(kind.mime).split('/')[0]=='image':
                # 顯示圖片
                img = mpimg.imread(file_path)  # 讀取和程式碼處於同一目錄下的 i
                # 此時 img 是一個 np.array 了，可以對它進行任意處理
                plt.imshow(img)  # 顯示圖片
                plt.axis('off')  # 不顯示座標軸
                plt.show()
                # cv2_imshow(im)
                # 圖片壓縮成像素300*300
                im = copy.deepcopy(cv2.imread(file_path))  # 讀取和程式碼處於同一目錄下的 i
                # 此時 im 是一個 np.array 了，可以對它進行任意處理
                dst = copy.deepcopy(cv2.resize(im, (300, 300), interpolation=cv2.INTER_AREA))  # resize
                dst = copy.deepcopy(dst[np.newaxis, :])  # 3維變4維
                dim4resizeimg.append(dst)
            else:
                dim4resizeimg.append(cutvideotoimage(file_path, str(input_number)+'image', str(input_number)+'data'))
        for i in range(2):
            #print(inputfile)
            input_file(file_path_storge[i], int(i+1))
        #print(dim4resizeimg,'dim4resizeimg')
        #print(inputfile,'inputfile')

        """
        此if else下影片截成圖片後
        都有resize以及呈現出圖片
        最後變成4維存入dim4resizeimg裡
        影片的4維有很多張圖片，圖片的4維有1張圖片
        dim4resizeimg長度為2，代表輸入的那兩個檔案
        """
        """
        if inputfile == 'image':
            load = u'./load_model_7.1_ring_predict_eig1/ttttt80080048048010.png'  # './11_6.png'
            load2 = u'./load_model_7.1_ring_predict_eig1/6C4804807.png'  # './5C480480331.png' # u'./15_4.png'
            dim4resizeimg = []
            for i in [load, load2]:
                # 顯示圖片
                img = mpimg.imread(i)  # 讀取和程式碼處於同一目錄下的 i
                # 此時 img 是一個 np.array 了，可以對它進行任意處理
                plt.imshow(img)  # 顯示圖片
                plt.axis('off')  # 不顯示座標軸
                plt.show()
                # cv2_imshow(im)
                # 圖片壓縮成像素300*300
                im = copy.deepcopy(cv2.imread(i))  # 讀取和程式碼處於同一目錄下的 i
                # 此時 im 是一個 np.array 了，可以對它進行任意處理
                dst = copy.deepcopy(cv2.resize(im, (300, 300), interpolation=cv2.INTER_AREA))  # resize
                dst = copy.deepcopy(dst[np.newaxis, :])  # 3維變4維
                dim4resizeimg.append(dst)
        else:
            load = u'./load_model_7.1_ring_predict_eig1/480480.mp4'
            load2 = u'./load_model_7.1_ring_predict_eig1/4scis640360.mp4'
            #load = '480480.mp4'
            #load2 = '4scis640360.mp4'
            # videoo(load,"./result_compressed.mp4")
            # videoo(load2,"./result_compresse.mp4")
            dim4resizeimg = [cutvideotoimage(load, '1image', 'data'), cutvideotoimage(load2, '2image', '2data')]
        """
        # dim4resizeimg=[檔案1四維, 檔案2四維]  影片與圖片的數量不同

        ringorstright1,_ = ring_or_stright(dim4resizeimg[0], inputfile[0])  # (第一個檔案(圖或影)) ringorstright1=0是直鏈，ringorstright1=1是環
        ringorstright2,_ = ring_or_stright(dim4resizeimg[1], inputfile[1])  # (第二個檔案(圖或影)) ringorstright2=0是直鏈，ringorstright2=1是環

        if ringorstright1 == 0:  # (第一個檔案(圖或影))若輸入圖片是直鏈
            HOMO1, LUMO1, b1, eigvals1, eigvecs1, bondorder1, bond1 = HOMOLUMOstraight(dim4resizeimg[0], inputfile[0],
                                                                                       'straight')  # 輸入4維檔案到函數裡
        else:  # (第一個檔案(圖或影))若輸入圖片是環
            HOMO1, LUMO1, b1, eigvals1, eigvecs1, bondorder1, bond1 = HOMOLUMOring(dim4resizeimg[0], inputfile[0], 'ring')

        if ringorstright2 == 0:  # (第二個檔案(圖或影))若輸入圖片是直鏈
            HOMO2, LUMO2, b2, eigvals2, eigvecs2, bondorder2, bond2 = HOMOLUMOstraight(dim4resizeimg[1], inputfile[1], 'straight')
        else:  # (第二個檔案(圖或影))若輸入圖片是直鏈
            HOMO2, LUMO2, b2, eigvals2, eigvecs2, bondorder2, bond2 = HOMOLUMOring(dim4resizeimg[1], inputfile[1], 'ring')
        #print(dim4resizeimg[0][0])
        #iface = gr.Interface(ring_or_stright, [gr.inputs.Image(dim4resizeimg[0][0]),'image'], outputs=["text",gr.outputs.Label(num_top_classes=2)])
        #iface.launch()
        """
        直鏈:
        HOMO1, LUMO1,b1=[array([[ 0.52112089],
               [ 0.23192061],
               [-0.41790651],
               [-0.41790651],
               [ 0.23192061],
               [ 0.52112089]])] [array([[-0.52112089],
               [ 0.23192061],
               [ 0.41790651],
               [-0.41790651],
               [-0.23192061],
               [ 0.52112089]])] 6 

        環:
        HOMO1, LUMO1,b1=[array([[-0.63245553],
               [-0.19543951],
               [ 0.51166727],
               [ 0.51166727],
               [-0.19543951]]), array([[ 0.07384113],
               [ 0.62020544],
               [ 0.30946691],
               [-0.42894437],
               [-0.57456911]])] [array([[ 0.63245553],
               [-0.51166727],
               [ 0.19543951],
               [ 0.19543951],
               [-0.51166727]]), array([[-0.03239904],
               [ 0.39747131],
               [-0.61072305],
               [ 0.59069934],
               [-0.34504856]])] 5
        """
        """
        #HOMO1LUMO2穩定還是LUMO1HOMO2穩定
        def totalenergy(mo,b,eigvals,eigvecs):
          for k in range(b):
            if np.all(mo[:, 0] == eigvecs[:, k]):
              energy=0
              for i in range(b):
                if (-eigvals[i])<=(-eigvals[k]):
                  energy+=(-eigvals[i]*2)
          return energy

        HOMOball=copy.deepcopy(b1)
        if (totalenergy(LUMO2,b2,eigvals2, eigvecs2)-totalenergy(HOMO1,b1,eigvals1, eigvecs1)) > (totalenergy(LUMO1,b1,eigvals1, eigvecs1)-totalenergy(HOMO2,b2,eigvals2, eigvecs2)):
          HOMOball=copy.deepcopy(b2)
        """


        # 步驟6前處理
        def layer3(MOO, b12, revers=False):
            """
            輸入[n*1鬼域,n*1鬼域](環)，或輸入輸入[n*1鬼域](直)，以及碳數，=>，n*20*1*3
            從[n*1鬼域,n*1鬼域]，拿出n*1鬼域，執行兩次迴圈，因為需要所有可能性被接到
            算出n隨機抽取兩個植，有幾種抽法，存入pn
            ppnn計算是否執行完pn次
            隨機選擇位置存入r1、r2
            看看是否要把軌域翻轉，並從n*1=>n*1*1
            建立通道(1個n*1*1=>3個n*1*1)
            經由r1、r2交換數值(3個n*1*1=>3個n*1*1)
            加入0(3個n*1*1=>3個20*1*1)
            合併成3通道((3個20*1*1=>20*1*3))
            加維度(20*1*3=>1*20*1*3)
            合併其他選擇的位置產生出的1*20*1*3變成n*20*1*3
            """
            rgbMO = []
            for MO in MOO:
                for rev in range(2):
                    if (rev == 1) and (revers == False):
                        break
                    # print(MO[::-1][:,:,np.newaxis],'mo')
                    if (rev == 1) and (revers == True) and np.allclose(MO[::-1], MO):
                        # print('ppp')
                        break
                    pn = int(x(b12) / ((x(b12 - 2)) * (x(2))))  # 每個軌域中選出兩個不同碳，有幾種分法
                    ppnn = 0
                    # n=int(10-(b12/2))
                    # c = int((20-b12)/2)
                    rr, r1, r2 = [], 0, 0
                    while ppnn != pn:  # 算次數，次數沒到繼續執行
                        # 隨機選位置
                        while ([r1, r2] in rr) or ([r2, r1] in rr) or (r1 == r2):  # 隨機產生r
                            r1 = random.randint(0, b12 - 1)
                            r2 = random.randint(0, b12 - 1)
                        rr.append([r1, r2])
                        if rev == 0:  # LUMO要轉才會讓不同方向配對到
                            # al=copy.deepcopy(MO[n:b12+n][:,:,np.newaxis])  #list(range((20)))20*1才使用
                            al = copy.deepcopy(MO[:, :, np.newaxis])
                            # if (len(rr)<2) and (revers==True):
                            # print(al,'al')
                        else:
                            # al=copy.deepcopy(MO[n:b12+n][::-1][:,:,np.newaxis])  # 20*1才使用
                            al = copy.deepcopy(MO[::-1][:, :, np.newaxis])
                            # if len(rr)<2:
                            # print(al,'alr')
                        # 建立通道
                        g = np.zeros(b12).reshape(b12, 1, 1)  # 第二層:綠色
                        bl = np.zeros(b12).reshape(b12, 1, 1)  # 第三層:藍色
                        # 交換數值
                        g[r1] = copy.deepcopy(al[r1])  # 紅色與綠色換數值
                        g[r2] = copy.deepcopy(al[r2])  # 紅色與綠色換數值
                        al[r1] = copy.deepcopy(0)  # 紅色與綠色換數值
                        al[r2] = copy.deepcopy(0)  # 紅色與綠色換數值
                        # 3個n*1變成3個20*1
                        gg = []
                        for i in [al, g, bl]:
                            # print(c,'c')
                            while len(i) < 20:
                                i = np.insert(i, 0, np.zeros(1), 0)
                                if len(i) < 20:
                                    i = np.insert(i, np.size(i, 0), np.zeros(1), 0)
                                else:
                                    break
                            gg.append(i)
                        al = copy.deepcopy(gg[0])
                        g = copy.deepcopy(gg[1])
                        bl = copy.deepcopy(gg[2])
                        # 3個20*1變成20*1*3
                        reg = np.append(al, g, axis=2)  # （：：在這裡合併）
                        regbl = np.append(reg, bl, axis=2)
                        # 20*1*3變成1*20*1*3
                        regbl = copy.deepcopy(regbl[np.newaxis, :])
                        # 1*20*1*3變成n*20*1*3
                        n = []
                        if rgbMO == []:
                            rgbMO = copy.deepcopy(regbl)
                            if rev == 1:
                                n.append((len(rgbMO) - 1))
                        else:
                            rgbMO = copy.deepcopy(np.concatenate((rgbMO, regbl), axis=0))
                            if rev == 1:
                                n.append((len(rgbMO) - 1))
                        ppnn += 1
            return rgbMO, n


        # HOMO1 是n*20*1*3
        HOMO1, Hn1 = layer3(HOMO1, b1)
        LUMO1, Ln1 = layer3(LUMO1, b1, revers=True)
        HOMO2, Hn2 = layer3(HOMO2, b2)
        LUMO2, Ln2 = layer3(LUMO2, b2, revers=True)
        #print(LUMO1[6], '6')
        # print(LUMO1[Ln1[0]],'Ln1[0]')

        b23 = []
        isringorstraight = []
        moleculebond = []
        for i in range(len(HOMO1)):
            for j in range(len(LUMO2)):
                reaction_rgb = np.append(HOMO1[i], LUMO2[j], axis=1)  # HOMOLUMO合併
                b23.append([b1, b2])
                isringorstraight.append([ringorstright1, ringorstright2])
                if j in Ln2:
                    moleculebond.append([bond1, bond2[::-1]])
                else:
                    moleculebond.append([bond1, bond2])
                reaction_rgb = copy.deepcopy(reaction_rgb[np.newaxis, :])
                if i == 0 and j == 0:
                    reaction_rgb12 = copy.deepcopy(reaction_rgb)
                else:
                    reaction_rgb12 = copy.deepcopy(np.concatenate([reaction_rgb12, reaction_rgb]))

        for i in range(len(HOMO2)):
            for j in range(len(LUMO1)):
                reaction_rgb = copy.deepcopy(np.append(HOMO2[i], LUMO1[j], axis=1))  # HOMOLUMO合併
                b23.append([b2, b1])
                isringorstraight.append([ringorstright2, ringorstright1])
                if j in Ln1:
                    moleculebond.append([bond2, bond1[::-1]])
                else:
                    moleculebond.append([bond2, bond1])
                reaction_rgb = copy.deepcopy(reaction_rgb[np.newaxis, :])
                reaction_rgb12 = copy.deepcopy(np.concatenate([reaction_rgb12, reaction_rgb]))
        #print(reaction_rgb12[5])
        """
        def HOMOorLUMOtobranchringbranch(lacktwonumber):
          Substituent=[]
          n=0
          for j in range(len(HOMOlacktwonumber)):  # HOMO中的各個數值的索引號
            # 蒐集支鏈與環的碳數
            if lacktwonumber[j]==0:
              #print(reaction_rgb12[i][:,0,1][HOMOc:HOMOc+b23[i][0]])
              #print(j, len(HOMOlacktwonumber))
              if n==0:
                Substituent.append((j))  # [2]
                #HOMO_first_bond_carbon_wavenumber_list.append(reaction_rgb12[i][:,0,1][HOMOc:HOMOc+b23[i][0]][j])
              else:
                Substituent.append((j-Substituent[0]+1))  # [2,4]
                Substituent.append((len(lacktwonumber)-j-1))  # [2,4,0]
                #HOMO_second_bond_carbon_wavenumber_list.append(reaction_rgb12[i][:,0,1][HOMOc:HOMOc+b23[i][0]][j]) # [支,產物環,支]
              n+=1
          return Substituent
        """


        def kfork(ij, b, bond, st):
            n = []
            ringzero = []
            for j in range(len(ij)):
                if ij[j] == 0:
                    n.append(j)
                    if len(n) != 4 and len(n) != 2 and len(n) != 1:
                        # print(n,'len(n)!=4')

                        oneSubstituent, rebond = 0, 0
                        if st == 0:
                            oneSubstituent, rebond = [n[0], len(n), (b - (len(n) + n[0]))], bond
                        n = [j]
                        if len(bond) > b and j < len(bond):
                            ringzero.append(j)
                    if len(n) == 4 or len(n) == 2:
                        # print(n,'len(n)==4')
                        if st == 1:
                            oneSubstituent = [len(n), (b - len(n))]
                            if len(bond) > b:
                                rebond = bond[n[0]:(n[0] + b)]
                                # print(rebond,'len(bond)>b')
                                break
                            else:
                                rebond = [element for lis in [bond[n[0]:], bond[:n[0]]] for element in lis]
                                # print(rebond,'else')
                                # print(bond,bond[n[0]:],bond[:n[0]],'ppp')
                                break
                        else:
                            oneSubstituent = [n[0], len(n), (b - (len(n) + n[0]))]
                            rebond = bond
                            # print(rebond,'bond')
                            break
                else:
                    if n != 0:
                        n.append(j)
            if (len(bond) > b) and (oneSubstituent == 0):
                oneSubstituent, rebond = [len(ringzero), (b - len(ringzero))], bond[ringzero[0]:(ringzero[0] + b)]
                #print(ij, bond, 'bondv')
            return oneSubstituent, rebond


        # 步驟6
        pair1 = []
        pair2 = []
        HOMOSubstituent, LUMOSubstituent, HOMObond, LUMObond = [], [], [], []
        HOMO_first_bond_carbon_wavenumber_list = copy.deepcopy([])
        HOMO_second_bond_carbon_wavenumber_list = copy.deepcopy([])
        LUMO_first_bond_carbon_wavenumber_list = copy.deepcopy([])
        LUMO_second_bond_carbon_wavenumber_list = copy.deepcopy([])
        """
        若預測會反應，則輸出HOMO中在產物當作取代基與環的部分[[X,X],[X,X,X]...]((環,取),(取,環,取)...)、輸出LUMO中在產物當作取代基與環的部分[[X,X],[X,X,X]...]
        [[2, 4], [4, 2], [2, 4], [4, 2],....
        [[2, 4, 2], [2, 4, 2], [2, 4, 2], [2, 4, 2],.....


        """
        model = tf.keras.models.load_model(u'./load_model_7.1_ring_predict_eig1/predict_position.h5')
        #print(len(reaction_rgb12), 'reaction_rgb12')
        for i in range(len(reaction_rgb12)):
            p = reaction_rgb12[i][np.newaxis, :]
            # print(p.shape)
            prediction = model.predict_classes(p)
            # 步驟7前處理
            if prediction[0] == 1:
                # print(prediction[0], p)
                # HOMOc = int((20-b23[i][0])/2)  # b23[i][0]是碳數，有可能是b2 or b1
                # HOMOlacktwonumber = reaction_rgb12[i][:,0,0][HOMOc:HOMOc+b23[i][0]]
                HOMOlacktwonumber = reaction_rgb12[i][:, 0, 0]
                while len(HOMOlacktwonumber) > b23[i][0]:
                    HOMOlacktwonumber = np.delete(HOMOlacktwonumber, 0)
                    if len(HOMOlacktwonumber) > b23[i][0]:
                        HOMOlacktwonumber = np.delete(HOMOlacktwonumber, -1)
                    else:
                        break
                # print(HOMOlacktwonumber)
                # HOMOlacktwonumber= [ 0.          0.62020544  0.30946691 -0.42894437  0.        ]...
                # oneHOMOSubstituent=HOMOorLUMOtobranchringbranch(HOMOlacktwonumber)
                # print(isringorstraight[i][0])
                # if isringorstraight[i][0]==1:
                # print(HOMOlacktwonumber,'前')
                zero4oncontactposition = [element for lis in [HOMOlacktwonumber, HOMOlacktwonumber] for element in lis]
                zero4oncontactbond = [element for lis in [moleculebond[i][0], moleculebond[i][0]] for element in lis]
                oneHOMOSubstituent, oneHOMObond = copy.deepcopy(
                    kfork(HOMOlacktwonumber, b23[i][0], moleculebond[i][0], isringorstraight[i][0]))
                if oneHOMOSubstituent == 0:
                    #print(zero4oncontactposition, 'zero4oncontactposition')
                    oneHOMOSubstituent, oneHOMObond = kfork(zero4oncontactposition, b23[i][0], zero4oncontactbond,
                                                            isringorstraight[i][0])
                # print(oneHOMObond,'後')
                # print(oneHOMOSubstituent,'後')
                HOMOSubstituent.append(
                    oneHOMOSubstituent)  # [[2, 4, 0], [2, 4, 0], [0, 4, 2], [0, 4, 2], [0, 4, 0], [0, 4, 0], [0, 4, 0]]
                HOMObond.append(oneHOMObond)
                #if oneHOMOSubstituent == 0:
                    #print(zero4oncontactposition, 'zero4oncontactposition2')
                    #print(zero4oncontactbond, 'zero4oncontactbond2')

                # LUMOc = int((20-b23[i][1])/2)  # b23[i][0]是碳數，有可能是b2 or b1
                # LUMOlacktwonumber = reaction_rgb12[i][:,1,0][LUMOc:LUMOc+b23[i][1]]
                LUMOlacktwonumber = reaction_rgb12[i][:, 1, 0]
                while len(LUMOlacktwonumber) > b23[i][1]:
                    LUMOlacktwonumber = np.delete(LUMOlacktwonumber, 0)
                    if len(LUMOlacktwonumber) > b23[i][1]:
                        LUMOlacktwonumber = np.delete(LUMOlacktwonumber, -1)
                    else:
                        break
                zero4oncontactposition = [element for lis in [LUMOlacktwonumber, LUMOlacktwonumber] for element in lis]
                zero4oncontactbond = [element for lis in [moleculebond[i][1], moleculebond[i][1]] for element in lis]
                oneLUMOSubstituent, oneLUMObond = copy.deepcopy(
                    kfork(LUMOlacktwonumber, b23[i][1], moleculebond[i][1], isringorstraight[i][1]))
                if oneLUMOSubstituent == 0:
                    #print(zero4oncontactposition, 'zero4oncontactposition')
                    oneLUMOSubstituent, oneLUMObond = kfork(zero4oncontactposition, b23[i][1], zero4oncontactbond,
                                                            isringorstraight[i][1])
                LUMOSubstituent.append(oneLUMOSubstituent)
                #if oneLUMOSubstituent[1] == 3:
                    #print(oneLUMOSubstituent, LUMOlacktwonumber, 'if oneLUMObond[1]==3:')

                LUMObond.append(oneLUMObond)
                #if oneLUMOSubstituent == 0:
                    #print(zero4oncontactposition, 'zero4oncontactposition2')
                    #print(zero4oncontactbond, 'zero4oncontactbond2')
        """
        print(len(LUMOSubstituent), 'lenLUMOSubstituent')
        print(LUMObond, 'LUMObond')
        print(LUMOSubstituent, 'LUMOSubstituent')  # 最後步驟要用到
        print(HOMOSubstituent)  # 最後步驟要用到
        print('HOMO_first_bond_carbon_wavenumber_list\n', HOMO_first_bond_carbon_wavenumber_list)
        print('HOMO_second_bond_carbon_wavenumber_list\n', HOMO_second_bond_carbon_wavenumber_list)
        print('LUMO_first_bond_carbon_wavenumber_list\n', LUMO_first_bond_carbon_wavenumber_list)
        print('LUMO_second_bond_carbon_wavenumber_list\n', LUMO_second_bond_carbon_wavenumber_list)
        """

        # print(len(LUMOSubstituent),'p',len(HOMO_first_bond_carbon_wavenumber_list))

        def subsmiles(Subs, MObond):
            #print(MObond, 'MObond')
            #print(Subs, 'Subs')
            amoleculeSubstituentssmiles = copy.deepcopy([])
            if len(Subs) == 3:
                for hj in range(3):
                    if hj == 1:
                        continue
                    if Subs[hj] == 0:
                        amoleculeSubstituentssmiles.append("")
                    else:
                        oneSubstituentsmiles = copy.deepcopy([])
                        for ij in range(Subs[hj]):
                            if ij == 0:
                                oneSubstituentsmiles.append('C')
                            else:
                                if hj == 0:
                                    n = -1
                                else:
                                    n = Subs[0] + Subs[1] - 1
                                if MObond[(ij + n)] == 2:
                                    oneSubstituentsmiles.append('=C')
                                if MObond[(ij + n)] == 1:
                                    oneSubstituentsmiles.append('C')
                        amoleculeSubstituentssmiles.append("".join(oneSubstituentsmiles))
            else:
                if Subs[1] == 0:
                    amoleculeSubstituentssmiles.append("")
                else:
                    oneSubstituentsmiles = copy.deepcopy([])
                    for ij in range(Subs[1]):
                        if ij == 0:
                            oneSubstituentsmiles.append('C')
                        else:
                            n = (Subs[0] - 1)
                            if MObond[(ij + n)] == 2:
                                oneSubstituentsmiles.append('=C')
                            if MObond[(ij + n)] == 1:
                                oneSubstituentsmiles.append('C')
                    oneSubstituentsmiles.append('2')
                    amoleculeSubstituentssmiles.append("2")
                    amoleculeSubstituentssmiles.append("".join(oneSubstituentsmiles))
            return amoleculeSubstituentssmiles


        ringssmiles, Substituentssmiles, repe, HOMOSubstituentnotrepe, LUMOSubstituentnotrepe = [], [], [], [], []
        wavenumber_multiply = copy.deepcopy([])
        ringpart = copy.deepcopy([])
        for i in range(len(LUMOSubstituent)):

            """
            # 可以有重複
            wavenumber_multiply.append([2*HOMO_first_bond_carbon_wavenumber_list[i]*LUMO_first_bond_carbon_wavenumber_list[i],2*HOMO_second_bond_carbon_wavenumber_list[i]*LUMO_second_bond_carbon_wavenumber_list[i]])
            HOMOSubstituentnotrepe.append(HOMOSubstituent[i])
            LUMOSubstituentnotrepe.append(LUMOSubstituent[i])
            """
            # 不能有重複
            """
            if [HOMOSubstituent[i],LUMOSubstituent[i]] not in repe:
              wavenumber_multiply.append([2*HOMO_first_bond_carbon_wavenumber_list[i]*LUMO_first_bond_carbon_wavenumber_list[i],2*HOMO_second_bond_carbon_wavenumber_list[i]*LUMO_second_bond_carbon_wavenumber_list[i]])
              repe.append([HOMOSubstituent[i],LUMOSubstituent[i]])
              HOMOSubstituentnotrepe.append(HOMOSubstituent[i])
              LUMOSubstituentnotrepe.append(LUMOSubstituent[i])
            else:
              continue
            """
            """
            # 不能有重複
            if [HOMO_first_bond_carbon_wavenumber_list[i],HOMO_second_bond_carbon_wavenumber_list[i],LUMO_first_bond_carbon_wavenumber_list[i],LUMO_second_bond_carbon_wavenumber_list[i]] not in repe:
              wavenumber_multiply.append([2*HOMO_first_bond_carbon_wavenumber_list[i]*LUMO_first_bond_carbon_wavenumber_list[i],2*HOMO_second_bond_carbon_wavenumber_list[i]*LUMO_second_bond_carbon_wavenumber_list[i]])
              repe.append([HOMO_first_bond_carbon_wavenumber_list[i],HOMO_second_bond_carbon_wavenumber_list[i],LUMO_first_bond_carbon_wavenumber_list[i],LUMO_second_bond_carbon_wavenumber_list[i]])
              HOMOSubstituentnotrepe.append(HOMOSubstituent[i])
              LUMOSubstituentnotrepe.append(LUMOSubstituent[i])
            else:
              continue
            """

            ring = copy.deepcopy([])

            # RR=copy.deepcopy([])
            if len(HOMOSubstituent[i]) == 3:
                kj = 1
                HOMOSone = 0
                HOMOStwo = 2
            else:
                kj, HOMOSone, HOMOStwo = 0, 1, 1
            if len(LUMOSubstituent[i]) == 3:
                jk, LUMOSone, LUMOStwo = 1, 0, 2
            else:
                jk, LUMOSone, LUMOStwo = 0, 1, 1
            for j in [HOMOSubstituent[i][kj], LUMOSubstituent[i][jk]]:
                # print(j,'j')
                if j == 2:
                    if len(ring) == 1:
                        ring.append('CC')
                        # RR.append(';R3;R4$|')
                    else:
                        ring.append('CC')
                        # RR.append('|$;R1;R2')
                elif j == 4:
                    if len(ring) == 1:
                        ring.append('CC=CC')
                        # RR.append(';R3;;;R4$|')
                    else:
                        ring.append('CC=CC')
                        # RR.append('|$;R1;;;R2')
                else:
                    #print([HOMOSubstituent[i], LUMOSubstituent[i]], '行程還得探數沒有4也沒有二')
                    if len(ring) == 1:
                        ringa = ['CC=', 'C']
                        # RRa = [';R3;',';R4$|']
                    else:
                        ringa = ['CC=', 'C']
                        # RRa = ['|$;R1;',';R2']
                    ringa.insert(3, 'C' * (j - 3))
                    ring.append("".join(ringa))
                    # RRa.insert(5,';'*(j-3))
                    # RR.append(RRa[0])
            # ring = copy.deepcopy(['C1=','C1'])
            # ringCnumber = HOMOSubstituent[i][1]+LUMOSubstituent[i][1]-2
            # ring.insert(1,'C'*ringCnumber)
            #print(ring, str([HOMOSubstituent[i][kj], LUMOSubstituent[i][jk]]))
            ringpart.append(ring)
            ring = copy.deepcopy("".join(ring))
            # RR = "".join(RR)
            ring = copy.deepcopy(ring[1:-1])
            # print(ring)
            ring = 'C1' + ring + 'C1'
            ringssmiles.append(ring)
            #print(ring, 'ring')

            # 儲存取代積碳數
            # amoleculeSubstituents=copy.deepcopy([HOMOSubstituent[i][HOMOSone],LUMOSubstituent[i][LUMOSone],LUMOSubstituent[i][LUMOStwo],HOMOSubstituent[i][HOMOStwo]])
            amoleculeSubstituents = copy.deepcopy([HOMOSubstituent[i], LUMOSubstituent[i]])
            amoleculebondd = copy.deepcopy([HOMObond[i], LUMObond[i], LUMObond[i], HOMObond[i]])
            molecule4Substituentssmiles = copy.deepcopy([])
            # 製作取代基smiles
            for j in range(2):
                aamoleculeSubstituentssmiles = subsmiles(amoleculeSubstituents[j], amoleculebondd[j])
                if j == 0:
                    molecule4Substituentssmiles.append(aamoleculeSubstituentssmiles)
                else:
                    molecule4Substituentssmiles = [molecule4Substituentssmiles[0][0], molecule4Substituentssmiles[0][1],
                                                   aamoleculeSubstituentssmiles[1], aamoleculeSubstituentssmiles[0]]
            """                                          
            for j in amoleculeSubstituents:
              if j==0:
                amoleculeSubstituentssmiles.append("")
              else:
                oneSubstituentsmiles=copy.deepcopy([])
                for ij in range(j):
                  if ij%2==0:
                    oneSubstituentsmiles.append("C")
                  else:
                    oneSubstituentsmiles.append('=C')
                amoleculeSubstituentssmiles.append("".join(oneSubstituentsmiles))
            """
            Substituentssmiles.append(molecule4Substituentssmiles)
            #print(amoleculeSubstituents, 'amoleculeSubstituents')
            # print(aamoleculeSubstituentssmiles,'aamoleculeSubstituentssmiles')
            #print(molecule4Substituentssmiles, 'molecule4Substituentssmiles')
        #print(Substituentssmiles)  # 最後步驟要用到
        #print(ringssmiles)  # 最後步驟要用到
        #print(ringpart)

        """

        # 產物穩定性排名
        print(wavenumber_multiply)
        wavenumber_multiply_rank = []
        for i in range(len(wavenumber_multiply)):
          n=copy.deepcopy(0)
          for j in range(len(wavenumber_multiply)):
            if (wavenumber_multiply[i][0]+wavenumber_multiply[i][1])<(wavenumber_multiply[j][0]+wavenumber_multiply[j][1]):
              n+=1
          if np.sign(wavenumber_multiply[i][0])<0 or np.sign(wavenumber_multiply[i][1])<0:
            wavenumber_multiply_rank.append([n,'unstable'])
          else:
            wavenumber_multiply_rank.append([n,'stable'])

        print(wavenumber_multiply_rank)
        print(len(wavenumber_multiply_rank))
        """
        # 步驟7
        # 已黑線條的方式呈現分子結構
        from rdkit.Chem.Draw import IPythonConsole
        #from rdkit.Chem import Draw
        #from rdkit.Chem.Draw import rdMolDraw2D
        import IPython
        """
        def oo(b):
            IPythonConsole.ipython_useSVG = True  # < set this to False if you want PNGs instead of SVGs

            def mol_with_atom_index(b):
                for atom in mol.GetAtoms():
                    atom.SetAtomMapNum(atom.GetIdx())
                return b

            display(b)
        """

        # 手性
        allchiral = copy.deepcopy([])
        nn = 0
        for i in range(len(Substituentssmiles)):
            n = 0
            for j in Substituentssmiles[i]:
                if len(j) != 0:  # 若某一邊含有取代基則有@@與@兩種情況，若含有兩個取代基，則有@@、@與@@、@@與@、@與@、@@四種情況，所以是2*2
                    if n == 0:
                        n += 2
                    else:
                        n *= 2  # n代表所有可能的手性個數
            nn += n
            norepeat = copy.deepcopy([])
            fourchiral = copy.deepcopy([])
            for k in range(n):
                while (fourchiral in norepeat) or (len(norepeat) == 0 and len(fourchiral) == 0):

                    fourchiral = copy.deepcopy([])
                    for j in Substituentssmiles[i]:
                        if len(j) == 0:
                            fourchiral.append('')
                        else:
                            random1 = copy.deepcopy(random.choice(['[C@@H]', '[C@H]']))
                            fourchiral.append(random1)
                # print(fourchiral,'fourchiral')
                # print(Substituentssmiles[i],'Substituentssmiles[i]')
                norepeat.append(fourchiral)  # norepeat是"單一"產物中所有可能手性的情況
            allchiral.append(norepeat)  # allchiral是"所有"產物中所有可能手性的情況

        motifyring = []
        ringname = []
        rank = []
        norankname = []
        #print(ringpart, 'ringpart')
        #print(ringssmiles, 'ringssmiles')
        #print(ring, 'ring')

        list_price_positoin_address = []  # https://blog.csdn.net/sinat_32651363/article/details/80940523
        for j in range(len(ringssmiles)):
            for i in ringssmiles[j]:
                address_index = [x for x in range(len(ringssmiles[j])) if ringssmiles[j][x] == 'C']
            list_price_positoin_address.append(address_index)
        #print(ringssmiles)
        #print(list_price_positoin_address)
        motifyring = []
        motifyringone = []
        for j in range(len(ringssmiles)):
            for k in range(len(allchiral[j])):  # for k in range(len(allchiral[j])):
                m = copy.deepcopy(ringssmiles[j])
                #print(m)
                #print(list_price_positoin_address[j])
                for i in range(4):
                    if Substituentssmiles[j][3 - i] == '':
                        continue
                    else:
                        # print(ringssmiles[j],list_price_positoin_address[j],len(list_price_positoin_address[j])-1)
                        if Substituentssmiles[j][3 - i] != '2':
                            branchh = '(' + Substituentssmiles[j][3 - i] + ')'
                        else:
                            branchh = Substituentssmiles[j][3 - i]
                        if i == 0:
                            if m[(list_price_positoin_address[j][
                                len(list_price_positoin_address[j]) - 1]) - 1] == '=':  # 若有雙劍代表不能接H，否則就有五個件與C連
                                cantfivebranch = allchiral[j][k][3 - i][:-2] + ']'
                                # print(cantfivebranch,'lko')
                            else:
                                cantfivebranch = allchiral[j][k][3 - i]
                            #print(m[:(list_price_positoin_address[j][len(list_price_positoin_address[j]) - 1])], cantfivebranch,
                                  #branchh, m[(list_price_positoin_address[j][len(list_price_positoin_address[j]) - 1] + 1):],
                                  #'i==0')
                            m = m[:(list_price_positoin_address[j][
                                len(list_price_positoin_address[j]) - 1])] + cantfivebranch + branchh + m[(
                                                                                                                      list_price_positoin_address[
                                                                                                                          j][
                                                                                                                          len(
                                                                                                                              list_price_positoin_address[
                                                                                                                                  j]) - 1] + 1):]  # C1CCC1 0 C1CC。 C1C=CCCC=CC1 0 C1C=CCCC=C。
                        if i == 1:
                            if m[(list_price_positoin_address[j][len(list_price_positoin_address[j]) - 1 - (
                                    ringpart[j][1].count('C') - 1)]) - 1] == '=':  # 若有雙劍代表不能接H，否則就有五個件與C連
                                cantfivebranch = allchiral[j][k][3 - i][:-2] + ']'
                                # print(cantfivebranch,'lko')
                            else:
                                cantfivebranch = allchiral[j][k][3 - i]
                            #print(m[:(list_price_positoin_address[j][
                                #len(list_price_positoin_address[j]) - 1 - (ringpart[j][1].count('C') - 1)])], cantfivebranch,
                                  #branchh, m[(list_price_positoin_address[j][len(list_price_positoin_address[j]) - 1 - (
                                            #ringpart[j][1].count('C') - 1)] + 1):], 'i==1')
                            m = m[:(list_price_positoin_address[j][len(list_price_positoin_address[j]) - 1 - (
                                        ringpart[j][1].count('C') - 1)])] + cantfivebranch + branchh + m[(
                                                                                                                     list_price_positoin_address[
                                                                                                                         j][len(
                                                                                                                         list_price_positoin_address[
                                                                                                                             j]) - 1 - (
                                                                                                                                        ringpart[
                                                                                                                                            j][
                                                                                                                                            1].count(
                                                                                                                                            'C') - 1)] + 1):]
                        if i == 2:
                            if m[(list_price_positoin_address[j][len(list_price_positoin_address[j]) - 1 - (
                                    ringpart[j][1].count('C') - 1) - 1]) - 1] == '=':  # 若有雙劍代表不能接H，否則就有五個件與C連
                                cantfivebranch = allchiral[j][k][3 - i][:-2] + ']'
                                # print(cantfivebranch,'lko')
                            else:
                                cantfivebranch = allchiral[j][k][3 - i]
                            #print(m[:(list_price_positoin_address[j][
                                #len(list_price_positoin_address[j]) - 1 - (ringpart[j][1].count('C') - 1) - 1])],
                                  #cantfivebranch, branchh, m[(list_price_positoin_address[j][
                                                                  #len(list_price_positoin_address[j]) - 1 - (
                                                                              #ringpart[j][1].count('C') - 1) - 1] + 1):],
                                  #'i==2')
                            m = m[:(list_price_positoin_address[j][len(list_price_positoin_address[j]) - 1 - (
                                        ringpart[j][1].count('C') - 1) - 1])] + cantfivebranch + branchh + m[(
                                                                                                                         list_price_positoin_address[
                                                                                                                             j][
                                                                                                                             len(
                                                                                                                                 list_price_positoin_address[
                                                                                                                                     j]) - 1 - (
                                                                                                                                         ringpart[
                                                                                                                                             j][
                                                                                                                                             1].count(
                                                                                                                                             'C') - 1) - 1] + 1):]  # C1CCC1 2。
                        if i == 3:
                            if m[(list_price_positoin_address[j][
                                len(list_price_positoin_address[j]) - 1 - (ringpart[j][1].count('C') - 1) - 1 - (
                                        ringpart[j][0].count('C') - 1)]) - 1] == '=':  # 若有雙劍代表不能接H，否則就有五個件與C連
                                cantfivebranch = allchiral[j][k][3 - i][:-2] + ']'
                                # print(cantfivebranch,'lko')
                            else:
                                cantfivebranch = allchiral[j][k][3 - i]
                            #print(m[:(list_price_positoin_address[j][
                                #len(list_price_positoin_address[j]) - 1 - (ringpart[j][1].count('C') - 1) - 1 - (
                                            #ringpart[j][0].count('C') - 1)])], cantfivebranch, branchh, m[(
                                                                                                                      #list_price_positoin_address[
                                                                                                                          #j][
                                                                                                                          #len(
                                                                                                                              #list_price_positoin_address[
                                                                                                                                  #j]) - 1 - (
                                                                                                                                      #ringpart[
                                                                                                                                          #j][
                                                                                                                                          #1].count(
                                                                                                                                          #'C') - 1) - 1 - (
                                                                                                                                      #ringpart[
                                                                                                                                       #   j][
                                                                                                                                        #  0].count(
                                                                                                                                         # 'C') - 1)] + 1):],
                                  #'i==3')
                            m = m[:(list_price_positoin_address[j][
                                len(list_price_positoin_address[j]) - 1 - (ringpart[j][1].count('C') - 1) - 1 - (
                                            ringpart[j][0].count('C') - 1)])] + cantfivebranch + branchh + m[(
                                                                                                                         list_price_positoin_address[
                                                                                                                             j][
                                                                                                                             len(
                                                                                                                                 list_price_positoin_address[
                                                                                                                                     j]) - 1 - (
                                                                                                                                         ringpart[
                                                                                                                                             j][
                                                                                                                                             1].count(
                                                                                                                                             'C') - 1) - 1 - (
                                                                                                                                         ringpart[
                                                                                                                                             j][
                                                                                                                                             0].count(
                                                                                                                                             'C') - 1)] + 1):]
                            # len(list_price_positoin_address[j])-1
                        # print(ringssmiles[j],i,m)
                #print(m)
                motifyring.append(m)
            motifyringone.append(m)
        """
        motify=[]
        for i in motifyringone:
          ba=copy.deepcopy(Chem.MolFromSmiles(i))
          ba=Chem.AddHs(ba)
          AllChem.EmbedMolecule(ba, AllChem.ETKDG())
          #AllChem.MMFFOptimizeMolecule(ba)
          motify.append(ba)
        n= len(motify)**0.5
        while len(motify)%math.ceil(n)!=0:
          n+=1
        n=math.ceil(n)
        m=len(motify)//n
        color_scheme = ['default']*len(motify)
        #cut=1
        #pdbdata=open('outfilepdb.txt', 'r').read()
        p = py3Dmol.view(width=n*200,height=m*100, viewergrid=(m,n), linked=False)
        for (nn, (i,j)),c in zip(enumerate([(a,b) for a in range(m) for b in range(n)]), color_scheme):
            b=copy.deepcopy(Chem.MolToMolBlock(motify[nn]))
            p.addModel(b,'sdf', viewer=(i,j))
            p.setStyle({'sphere':{'scale':.3},'stick':{}}, viewer=(i,j)) # cartoon  stick  sphere  {'sphere':{'scale':.3},'stick':{}}
            p.addLabel(motifyringone[nn],{'position': {'x':-1.89, 'y':-3, 'z':0.35}, 'backgroundColor': 0x800080, 'backgroundOpacity': 0.2}, viewer=(i,j))
        p.zoomTo()
        p.show()
        print(m,n) 
        #p.png()
        """

        momo = math.ceil(len(motifyringone) / 16)
        for k in range(16):
            if momo * (k + 1) > len(motifyringone):
                motimoti = copy.deepcopy(motifyringone[momo * k:])
            else:
                #print(momo * k, (momo * k + momo))
                motimoti = copy.deepcopy(motifyringone[momo * k:(momo * k + momo)])
            motify = []
            for i in motimoti:
                ba = copy.deepcopy(Chem.MolFromSmiles(i))
                ba = Chem.AddHs(ba)
                Chem.rdmolops.RemoveStereochemistry(ba)  # https://github.com/rdkit/rdkit/issues/1433
                AllChem.AssignAtomChiralTagsFromStructure(ba,confId=-1,replaceExistingTags=True)
                AllChem.EmbedMolecule(ba,maxAttempts=10000)  #, AllChem.ETKDG()
                AllChem.UFFOptimizeMolecule(ba)
                #AllChem.MMFFOptimizeMolecule(ba)
                motify.append(ba)
            n = len(motify) ** 0.5
            if n==0:
                continue
            while len(motify) % math.ceil(n) != 0:
                n += 1
            n = math.ceil(n)
            m = len(motify) // n
            color_scheme = ['default'] * len(motify)
            # cut=1
            # pdbdata=open('outfilepdb.txt', 'r').read()
            p = py3Dmol.view(width=n * 200, height=m * 100, viewergrid=(m, n), linked=False)
            for (nn, (i, j)), c in zip(enumerate([(a, b) for a in range(m) for b in range(n)]), color_scheme):
                b = copy.deepcopy(Chem.MolToMolBlock(motify[nn]))
                p.addModel(b, 'sdf', viewer=(i, j))
                p.setStyle({'sphere': {'scale': .3}, 'stick': {}},
                           viewer=(i, j))  # cartoon  stick  sphere  {'sphere':{'scale':.3},'stick':{}}
                p.addLabel(motimoti[nn], {'position': {'x': -1.89, 'y': -3, 'z': 0.35}, 'backgroundColor': 0x800080,
                                          'backgroundOpacity': 0.2}, viewer=(i, j))
            p.zoomTo()
            p.show()
            #print(m, n)
        # p.png()

        """
        p = py3Dmol.view(width=400,height=400)
        def drawit(m,p,confId=-1):
            mb = Chem.MolToMolBlock(m,confId=confId)
            p.removeAllModels()
            p.addModel(mb,'sdf')
            p.setStyle({'stick':{}})
            p.setBackgroundColor('0xeeeeee')
            p.zoomTo()
            return p.show()
        from ipywidgets import interact, interactive, fixed
        for i in range(len(motifyringone)):
            #smile_fromMolecule = Chem.MolToSmiles(motifyringone[i])
            m = Chem.MolFromSmiles(motifyringone[i])
            #print(smile_fromMolecule)
            m = Chem.AddHs(m)
            AllChem.EmbedMultipleConfs(m,randomSeed=0xf00d,useExpTorsionAnglePrefs=True, useBasicKnowledge=True)
            interact(drawit, m=fixed(m),p=fixed(p))
        """

button_send = widgets.Button(
                description='up lod image',
                tooltip='Send',
                style={'description_width': 'initial'}
            )
"""
button_send2 = widgets.Button(
                description='clear_output',
                tooltip='Send',
                style={'description_width': 'initial'}
            )
"""

button_send.on_click(main)
#button_send.on_click(on_button_clicked)
        

page = widgets.HBox([button_send,out])
display(page)